In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,67,2024-03-16,1,113.757576,96.706061,0.557879,11.495455,23.713636,0.181924,116.796970,...,11.003030,24.684848,0.203788,114.146970,0,3,5,3,6,2024
1,67,2024-03-16,-1,111.833333,96.498485,0.536712,10.857576,24.596970,0.177273,115.316667,...,11.058209,23.143284,0.180806,121.259701,0,3,5,3,3,2024
2,67,2024-03-16,1,107.106061,97.033333,0.527667,12.024242,21.507576,0.165742,110.113636,...,10.000000,24.690909,0.230515,118.216667,0,3,5,3,5,2024
3,67,2024-03-16,-1,107.712121,96.490909,0.507000,12.648485,26.992424,0.193091,110.319697,...,11.436923,25.312308,0.209723,118.660000,0,3,5,2,25,2024
4,67,2024-03-16,1,120.772727,99.045455,0.577030,10.460606,20.736364,0.202470,121.189394,...,12.932836,22.823881,0.186060,108.007463,0,3,5,2,21,2024
5,67,2024-03-16,-2,114.227273,102.419697,0.538697,11.763636,19.692424,0.173030,111.336364,...,10.568657,24.294030,0.188299,115.288060,0,3,5,2,30,2024
6,66,2024-03-16,-1,118.246154,99.175385,0.553015,12.033846,27.204615,0.183154,118.133846,...,11.832836,19.517910,0.211194,116.200000,0,3,5,3,10,2024
7,67,2024-03-16,1,113.196970,96.565152,0.559530,12.528788,23.354545,0.217818,116.487879,...,12.855385,28.716923,0.209877,117.578462,0,3,5,4,18,2024
8,67,2024-03-16,2,112.151515,94.892424,0.535788,11.204545,29.610606,0.197515,118.215152,...,11.195385,23.761538,0.170600,118.469231,0,3,5,2,20,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-16,CLE,HOU,1.505051,2.640000,1
1,2024-03-16,BRK,IND,4.100000,1.253165,0
2,2024-03-16,CHO,PHI,5.100000,1.181818,0
3,2024-03-16,POR,NOP,7.500000,1.100000,0
4,2024-03-16,OKC,MEM,1.200000,4.800000,1
5,2024-03-16,WAS,CHI,4.600000,1.212766,0
6,2024-03-16,GSW,LAL,2.100000,1.769231,0
7,2024-03-16,MIN,UTA,1.294118,3.700000,1
8,2024-03-16,NYK,SAC,2.360000,1.617284,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-16,CLE,HOU,1.505051,2.640000,1
1,2024-03-16,BRK,IND,4.100000,1.253165,0
2,2024-03-16,CHO,PHI,5.100000,1.181818,0
3,2024-03-16,POR,NOP,7.500000,1.100000,0
4,2024-03-16,OKC,MEM,1.200000,4.800000,1
5,2024-03-16,WAS,CHI,4.600000,1.212766,0
6,2024-03-16,GSW,LAL,2.100000,1.769231,0
7,2024-03-16,MIN,UTA,1.294118,3.700000,0
8,2024-03-16,NYK,SAC,2.360000,1.617284,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-16,CLE,HOU,1.505051,2.640000,0
1,2024-03-16,BRK,IND,4.100000,1.253165,0
2,2024-03-16,CHO,PHI,5.100000,1.181818,0
3,2024-03-16,POR,NOP,7.500000,1.100000,0
4,2024-03-16,OKC,MEM,1.200000,4.800000,1
5,2024-03-16,WAS,CHI,4.600000,1.212766,0
6,2024-03-16,GSW,LAL,2.100000,1.769231,0
7,2024-03-16,MIN,UTA,1.294118,3.700000,0
8,2024-03-16,NYK,SAC,2.360000,1.617284,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-16,CLE,HOU,1.505051,2.640000,0
1,2024-03-16,BRK,IND,4.100000,1.253165,0
2,2024-03-16,CHO,PHI,5.100000,1.181818,0
3,2024-03-16,POR,NOP,7.500000,1.100000,0
4,2024-03-16,OKC,MEM,1.200000,4.800000,1
5,2024-03-16,WAS,CHI,4.600000,1.212766,0
6,2024-03-16,GSW,LAL,2.100000,1.769231,0
7,2024-03-16,MIN,UTA,1.294118,3.700000,0
8,2024-03-16,NYK,SAC,2.360000,1.617284,1


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-16,CLE,HOU,1.505051,2.640000,1.0
1,2024-03-16,BRK,IND,4.100000,1.253165,0.0
2,2024-03-16,CHO,PHI,5.100000,1.181818,0.0
3,2024-03-16,POR,NOP,7.500000,1.100000,0.0
4,2024-03-16,OKC,MEM,1.200000,4.800000,1.0
5,2024-03-16,WAS,CHI,4.600000,1.212766,0.0
6,2024-03-16,GSW,LAL,2.100000,1.769231,0.0
7,2024-03-16,MIN,UTA,1.294118,3.700000,1.0
8,2024-03-16,NYK,SAC,2.360000,1.617284,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-16,CLE,HOU,1.505051,2.640000,1.0
1,2024-03-16,BRK,IND,4.100000,1.253165,0.0
2,2024-03-16,CHO,PHI,5.100000,1.181818,0.0
3,2024-03-16,POR,NOP,7.500000,1.100000,0.0
4,2024-03-16,OKC,MEM,1.200000,4.800000,1.0
5,2024-03-16,WAS,CHI,4.600000,1.212766,0.0
6,2024-03-16,GSW,LAL,2.100000,1.769231,0.0
7,2024-03-16,MIN,UTA,1.294118,3.700000,1.0
8,2024-03-16,NYK,SAC,2.360000,1.617284,0.0
